In [53]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras import applications
import tensorflow as tf
from tqdm import tqdm
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import random
from shutil import copyfile
import os
import gc
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
#creating train image generator
train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=(0.95,0.95),
    horizontal_flip=True,
    vertical_flip=False,
    dtype=tf.float32,
)
#creating validation image generator
valid = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=(0.95,0.95),
    horizontal_flip=True,
    vertical_flip=False,
    dtype=tf.float32,
)

In [42]:
#Loading train image dataset
train_data = train.flow_from_directory(
    './dataset/train/',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    seed = 619
)

#Loading validation image dataset
valid_data = train.flow_from_directory(
    './dataset/val/',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    seed = 619
)

Found 12753 images belonging to 104 classes.
Found 3712 images belonging to 104 classes.


In [43]:
#checking class indices
train_data.class_indices

{'alpine sea holly': 0,
 'anthurium': 1,
 'artichoke': 2,
 'azalea': 3,
 'balloon flower': 4,
 'barberton daisy': 5,
 'bee balm': 6,
 'bird of paradise': 7,
 'bishop of llandaff': 8,
 'black-eyed susan': 9,
 'blackberry lily': 10,
 'blanket flower': 11,
 'bolero deep blue': 12,
 'bougainvillea': 13,
 'bromelia': 14,
 'buttercup': 15,
 'californian poppy': 16,
 'camellia': 17,
 'canna lily': 18,
 'canterbury bells': 19,
 'cape flower': 20,
 'carnation': 21,
 'cautleya spicata': 22,
 'clematis': 23,
 "colt's foot": 24,
 'columbine': 25,
 'common dandelion': 26,
 'common tulip': 27,
 'corn poppy': 28,
 'cosmos': 29,
 'cyclamen ': 30,
 'daffodil': 31,
 'daisy': 32,
 'desert-rose': 33,
 'fire lily': 34,
 'foxglove': 35,
 'frangipani': 36,
 'fritillary': 37,
 'garden phlox': 38,
 'gaura': 39,
 'gazania': 40,
 'geranium': 41,
 'giant white arum lily': 42,
 'globe thistle': 43,
 'globe-flower': 44,
 'grape hyacinth': 45,
 'great masterwort': 46,
 'hard-leaved pocket orchid': 47,
 'hibiscus': 4

In [44]:
#specifing image dimensions
width,height = 224,224

In [45]:
#Transfer learning using VGG16 imagenet model
#Loading VGG16 model without top layer with custom input size
base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(width,height,3))


In [46]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [47]:
#freezing trainable parameters of base model
base_model.trainable=False

In [48]:
#Adding new layers to base model
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a 104 unit (categorical classification)
outputs = keras.layers.Dense(104)(x)
model = keras.Model(inputs, outputs)

In [49]:
#compiling the model
model.compile(loss=keras.losses.categorical_crossentropy,metrics=keras.metrics.categorical_accuracy)

In [56]:
#fitting the model 
model.fit(train_data,batch_size=32,steps_per_epoch=24,validation_data=valid_data,validation_steps=8,epochs=20)

Epoch 1/20
24/24 [==============================] - 9s 389ms/step - loss: 9.1133 - categorical_accuracy: 0.0495 - val_loss: 8.0320 - val_categorical_accuracy: 0.0352
Epoch 2/20
24/24 [==============================] - 9s 388ms/step - loss: 9.1843 - categorical_accuracy: 0.0638 - val_loss: 10.4783 - val_categorical_accuracy: 0.0547
Epoch 3/20
24/24 [==============================] - 9s 386ms/step - loss: 9.3667 - categorical_accuracy: 0.0521 - val_loss: 10.6711 - val_categorical_accuracy: 0.0352
Epoch 4/20
24/24 [==============================] - 9s 384ms/step - loss: 9.0090 - categorical_accuracy: 0.0482 - val_loss: 10.2562 - val_categorical_accuracy: 0.0664
Epoch 5/20
24/24 [==============================] - 9s 396ms/step - loss: 9.0147 - categorical_accuracy: 0.0430 - val_loss: 9.9052 - val_categorical_accuracy: 0.0625
Epoch 6/20
24/24 [==============================] - 9s 390ms/step - loss: 9.4509 - categorical_accuracy: 0.0612 - val_loss: 9.3146 - val_categorical_accuracy: 0.0547
E

In [54]:
#custom model 
num_classes = 104
custom_model = Sequential()
custom_model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(224, 224, 3)))
custom_model.add(BatchNormalization())
custom_model.add(MaxPool2D((2, 2), strides=2, padding="same"))
custom_model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
custom_model.add(Dropout(0.2))
custom_model.add(BatchNormalization())
custom_model.add(MaxPool2D((2, 2), strides=2, padding="same"))
custom_model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
custom_model.add(BatchNormalization())
custom_model.add(MaxPool2D((2, 2), strides=2, padding="same"))
custom_model.add(Flatten())
custom_model.add(Dense(units=512, activation="relu"))
custom_model.add(Dropout(0.3))
custom_model.add(Dense(units=num_classes, activation="softmax"))

In [55]:
custom_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 75)      2100      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 75)      300       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 75)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 50)      33800     
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 50)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 50)      200       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 50)       

In [57]:
custom_model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

In [60]:
history = custom_model.fit(train_data,batch_size=32,steps_per_epoch=400,validation_data=valid_data,validation_steps=120,epochs=50)

Epoch 1/50
400/400 [==============================] - 153s 382ms/step - loss: 3.8725 - accuracy: 0.1333 - val_loss: 4.1342 - val_accuracy: 0.1026
Epoch 2/50
400/400 [==============================] - 156s 389ms/step - loss: 3.7515 - accuracy: 0.1539 - val_loss: 3.6618 - val_accuracy: 0.1698
Epoch 3/50
400/400 [==============================] - 154s 384ms/step - loss: 3.7036 - accuracy: 0.1661 - val_loss: 4.0406 - val_accuracy: 0.1648
Epoch 4/50
400/400 [==============================] - 152s 380ms/step - loss: 3.6103 - accuracy: 0.1786 - val_loss: 3.4144 - val_accuracy: 0.2005
Epoch 5/50
400/400 [==============================] - 152s 381ms/step - loss: 3.5346 - accuracy: 0.1903 - val_loss: 4.0714 - val_accuracy: 0.1641
Epoch 6/50
400/400 [==============================] - 153s 383ms/step - loss: 3.5129 - accuracy: 0.1885 - val_loss: 3.3667 - val_accuracy: 0.1956
Epoch 7/50
400/400 [==============================] - 153s 383ms/step - loss: 3.4109 - accuracy: 0.2145 - val_loss: 3.3960 -

In [61]:
custom_model.save('flower_classifer')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: flower_classifer/assets


In [64]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training acc')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

NameError: name 'history' is not defined